# cs 5644 final project experiment 2b: predicting individual company prices

In [1]:
%matplotlib inline
import math
import pandas as pd
from pandas.plotting import lag_plot
from IPython.display import display
from sklearn import preprocessing
from sklearn import linear_model, neighbors
from sklearn import tree
from sklearn import model_selection
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor


import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# read in data

In [2]:
df_index = pd.read_csv('sp500_index.csv')
df_stocks = pd.read_csv('sp500_stocks.csv')
df_companies = pd.read_csv('sp500_companies.csv')

### create single data frame

had to convert date columns to datetime objects to merge them for some reason

In [3]:
df_stocks['Date'] = pd.to_datetime(df_stocks['Date'], utc=True, errors='coerce')
df_stocks['Date'] = pd.DatetimeIndex(df_stocks['Date']).date

df_index['Date'] = pd.to_datetime(df_index['Date'], utc=True)
df_index['Date'] = pd.DatetimeIndex(df_index['Date']).date

In [4]:
df = df_companies.merge(df_stocks, on='Symbol', how='inner')

In [5]:
df = df.merge(df_index, on='Date')
df.head()

,Exchange,Symbol,Shortname,Longname,Sector,Industry,Currentprice,Marketcap,Ebitda,Revenuegrowth,...,Longbusinesssummary,Weight,Date,Adj Close,Close,High,Low,Open,Volume,S&P500
0,NMS,AAPL,Apple Inc.,Apple Inc.,Technology,Consumer Electronics,140.42,2256661643264,1.295570e+11,0.019,...,"Apple Inc. designs, manufactures, and markets ...",0.070093,2012-10-11,19.372286,22.432142,23.114286,22.432142,23.089287,546081200.0,1432.84
1,NMS,MSFT,Microsoft Corporation,Microsoft Corporation,Technology,Software—Infrastructure,229.25,1721204342784,9.798300e+10,0.124,...,"Microsoft Corporation develops, licenses, and ...",0.053462,2012-10-11,23.988226,28.950001,29.250000,28.870001,29.219999,41488500.0,1432.84
2,NMS,GOOGL,Alphabet Inc.,Alphabet Inc.,Communication Services,Internet Content & Information,97.86,1281926889472,9.688700e+10,0.126,...,Alphabet Inc. provides various products and pl...,0.039818,2012-10-11,18.805805,18.805805,18.981482,18.776026,18.841341,95260644.0,1432.84
3,NMS,GOOG,Alphabet Inc.,Alphabet Inc.,Communication Services,Internet Content & Information,98.71,1281502609408,9.688700e+10,0.126,...,Alphabet Inc. provides various products and pl...,0.039804,2012-10-11,18.716845,18.716845,18.891689,18.687206,18.752213,95713418.0,1432.84
4,NMS,AMZN,"Amazon.com, Inc.","Amazon.com, Inc.",Consumer Cyclical,Internet Retail,113.67,1158024396800,5.262000e+10,0.072,...,"Amazon.com, Inc. engages in the retail sale of...",0.035969,2012-10-11,12.211000,12.211000,12.465000,12.094500,12.400000,68946000.0,1432.84


# preprocessing

### drop irrelevant columns

In [6]:
df.drop(labels=['Shortname', 'Longname', 'City', 'Longbusinesssummary', 'Date'], axis=1, inplace=True)

### one hot encode

In [7]:
ohe = preprocessing.OneHotEncoder(dtype=int, sparse=False, handle_unknown="ignore")

In [8]:
data = ohe.fit_transform(df[['Exchange', 'Sector', 'Industry', 'State', 'Country']])

In [9]:
cats = pd.DataFrame(data, columns=ohe.get_feature_names())

/Users/caitlinhermann/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [10]:
snp_df = pd.concat([cats, df], axis=1)

In [11]:
snp_df.drop(columns=['Exchange', 'Sector', 'Industry', 'State', 'Country'], inplace=True)

In [12]:
snp_df

,x0_BTS,x0_NGM,x0_NMS,x0_NYQ,x1_Basic Materials,x1_Communication Services,x1_Consumer Cyclical,x1_Consumer Defensive,x1_Energy,x1_Financial Services,...,Revenuegrowth,Fulltimeemployees,Weight,Adj Close,Close,High,Low,Open,Volume,S&P500
0,0,0,1,0,0,0,0,0,0,0,...,0.019,154000.0,0.070093,19.372286,22.432142,23.114286,22.432142,23.089287,546081200.0,1432.84
1,0,0,1,0,0,0,0,0,0,0,...,0.124,221000.0,0.053462,23.988226,28.950001,29.250000,28.870001,29.219999,41488500.0,1432.84
2,0,0,1,0,0,1,0,0,0,0,...,0.126,174014.0,0.039818,18.805805,18.805805,18.981482,18.776026,18.841341,95260644.0,1432.84
3,0,0,1,0,0,1,0,0,0,0,...,0.126,174014.0,0.039804,18.716845,18.716845,18.891689,18.687206,18.752213,95713418.0,1432.84
4,0,0,1,0,0,0,1,0,0,0,...,0.072,1523000.0,0.035969,12.211000,12.211000,12.465000,12.094500,12.400000,68946000.0,1432.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1244920,0,0,0,1,0,0,0,0,0,0,...,0.041,9300.0,0.000186,23.670000,23.670000,24.400000,23.650000,24.110001,1255555.0,3612.39
1244921,0,0,1,0,0,0,0,1,0,0,...,-0.065,32000.0,0.000186,14.070000,14.070000,14.210000,13.950000,14.020000,2551003.0,3612.39
1244922,0,0,0,1,0,0,0,0,0,0,...,0.741,22092.0,0.000162,41.180000,41.180000,41.910000,40.884998,41.680000,844974.0,3612.39
1244923,0,0,0,1,0,0,1,0,0,0,...,167.368,34700.0,0.000154,11.910000,11.910000,12.985000,11.500000,12.820000,28123604.0,3612.39


### check number of missing values

In [13]:
pd.options.display.max_columns = None
pd.options.display.max_rows = 250

In [14]:
percent_missing = snp_df.isnull().sum() * 100 / len(snp_df)
missing_value_df = pd.DataFrame({'column_name': snp_df.columns,
'percent_missing': percent_missing})

missing_value_df

,column_name,percent_missing
x0_BTS,x0_BTS,0.000000
x0_NGM,x0_NGM,0.000000
x0_NMS,x0_NMS,0.000000
x0_NYQ,x0_NYQ,0.000000
x1_Basic Materials,x1_Basic Materials,0.000000
x1_Communication Services,x1_Communication Services,0.000000
x1_Consumer Cyclical,x1_Consumer Cyclical,0.000000
x1_Consumer Defensive,x1_Consumer Defensive,0.000000
x1_Energy,x1_Energy,0.000000
x1_Financial Services,x1_Financial Services,0.000000


### drop missing values

In [15]:
snp_df.dropna(inplace=True)
snp_df

,x0_BTS,x0_NGM,x0_NMS,x0_NYQ,x1_Basic Materials,x1_Communication Services,x1_Consumer Cyclical,x1_Consumer Defensive,x1_Energy,x1_Financial Services,x1_Healthcare,x1_Industrials,x1_Real Estate,x1_Technology,x1_Utilities,x1_nan,x2_Advertising Agencies,x2_Aerospace & Defense,x2_Agricultural Inputs,x2_Airlines,x2_Apparel Manufacturing,x2_Apparel Retail,x2_Asset Management,x2_Auto & Truck Dealerships,x2_Auto Manufacturers,x2_Auto Parts,x2_Banks—Diversified,x2_Banks—Regional,x2_Beverages—Brewers,x2_Beverages—Non-Alcoholic,x2_Beverages—Wineries & Distilleries,x2_Biotechnology,x2_Building Materials,x2_Building Products & Equipment,x2_Business Equipment & Supplies,x2_Capital Markets,x2_Chemicals,x2_Communication Equipment,x2_Computer Hardware,x2_Confectioners,x2_Conglomerates,x2_Consulting Services,x2_Consumer Electronics,x2_Copper,x2_Credit Services,x2_Diagnostics & Research,x2_Discount Stores,x2_Drug Manufacturers—General,x2_Drug Manufacturers—Specialty & Generic,x2_Electronic Components,x2_Electronic Gaming & Multimedia,x2_Engineering & Construction,x2_Entertainment,x2_Farm & Heavy Construction Machinery,x2_Farm Products,x2_Financial Data & Stock Exchanges,x2_Food Distribution,x2_Footwear & Accessories,"x2_Furnishings, Fixtures & Appliances",x2_Gold,x2_Grocery Stores,x2_Healthcare Plans,x2_Home Improvement Retail,x2_Household & Personal Products,x2_Industrial Distribution,x2_Information Technology Services,x2_Insurance Brokers,x2_Insurance—Diversified,x2_Insurance—Life,x2_Insurance—Property & Casualty,x2_Insurance—Reinsurance,x2_Insurance—Specialty,x2_Integrated Freight & Logistics,x2_Internet Content & Information,x2_Internet Retail,x2_Leisure,x2_Lodging,x2_Medical Care Facilities,x2_Medical Devices,x2_Medical Distribution,x2_Medical Instruments & Supplies,x2_Oil & Gas E&P,x2_Oil & Gas Equipment & Services,x2_Oil & Gas Integrated,x2_Oil & Gas Midstream,x2_Oil & Gas Refining & Marketing,x2_Packaged Foods,x2_Packaging & Containers,x2_Personal Services,x2_Pharmaceutical Retailers,x2_REIT—Diversified,x2_REIT—Healthcare Facilities,x2_REIT—Hotel & Motel,x2_REIT—Industrial,x2_REIT—Office,x2_REIT—Residential,x2_REIT—Retail,x2_REIT—Specialty,x2_Railroads,x2_Real Estate Services,x2_Rental & Leasing Services,x2_Residential Construction,x2_Resorts & Casinos,x2_Restaurants,x2_Scientific & Technical Instruments,x2_Security & Protection Services,x2_Semiconductor Equipment & Materials,x2_Semiconductors,x2_Software—Application,x2_Software—Infrastructure,x2_Solar,x2_Specialty Business Services,x2_Specialty Chemicals,x2_Specialty Industrial Machinery,x2_Specialty Retail,x2_Staffing & Employment Services,x2_Steel,x2_Telecom Services,x2_Tobacco,x2_Tools & Accessories,x2_Travel Services,x2_Trucking,x2_Utilities—Diversified,x2_Utilities—Independent Power Producers,x2_Utilities—Regulated Electric,x2_Utilities—Regulated Gas,x2_Utilities—Regulated Water,x2_Utilities—Renewable,x2_Waste Management,x2_nan,x3_AL,x3_AR,x3_AZ,x3_CA,x3_CO,x3_CT,x3_DC,x3_DE,x3_FL,x3_GA,x3_IA,x3_ID,x3_IL,x3_IN,x3_KY,x3_LA,x3_MA,x3_MD,x3_ME,x3_MI,x3_MN,x3_MO,x3_NC,x3_NE,x3_NJ,x3_NV,x3_NY,x3_OH,x3_OK,x3_OR,x3_PA,x3_RI,x3_TN,x3_TX,x3_UT,x3_VA,x3_WA,x3_WI,x3_nan,x4_Bermuda,x4_Ireland,x4_Israel,x4_Netherlands,x4_Switzerland,x4_United Kingdom,x4_United States,x4_nan,Symbol,Currentprice,Marketcap,Ebitda,Revenuegrowth,Fulltimeemployees,Weight,Adj Close,Close,High,Low,Open,Volume,S&P500
0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,AAPL,140.42,2256661643264,1.295570e+11,0.019,154000.0,0.070093,19.372286,22.432142,23.114286,22.432142,23.089287,546081200.0,1432.84
1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

## separate x and y values
### + drop additional predictor columns

In [16]:
snp_xvalues = snp_df.drop(["Adj Close", "Close", "High", "Low", "Open", "Symbol"], axis=1)
snp_target = snp_df["Adj Close"]

# 

# 

# GOOG stock

In [46]:
goog_df = snp_df[snp_df['Symbol'] == 'GOOG']
goog_df

,x0_BTS,x0_NGM,x0_NMS,x0_NYQ,x1_Basic Materials,x1_Communication Services,x1_Consumer Cyclical,x1_Consumer Defensive,x1_Energy,x1_Financial Services,x1_Healthcare,x1_Industrials,x1_Real Estate,x1_Technology,x1_Utilities,x1_nan,x2_Advertising Agencies,x2_Aerospace & Defense,x2_Agricultural Inputs,x2_Airlines,x2_Apparel Manufacturing,x2_Apparel Retail,x2_Asset Management,x2_Auto & Truck Dealerships,x2_Auto Manufacturers,x2_Auto Parts,x2_Banks—Diversified,x2_Banks—Regional,x2_Beverages—Brewers,x2_Beverages—Non-Alcoholic,x2_Beverages—Wineries & Distilleries,x2_Biotechnology,x2_Building Materials,x2_Building Products & Equipment,x2_Business Equipment & Supplies,x2_Capital Markets,x2_Chemicals,x2_Communication Equipment,x2_Computer Hardware,x2_Confectioners,x2_Conglomerates,x2_Consulting Services,x2_Consumer Electronics,x2_Copper,x2_Credit Services,x2_Diagnostics & Research,x2_Discount Stores,x2_Drug Manufacturers—General,x2_Drug Manufacturers—Specialty & Generic,x2_Electronic Components,x2_Electronic Gaming & Multimedia,x2_Engineering & Construction,x2_Entertainment,x2_Farm & Heavy Construction Machinery,x2_Farm Products,x2_Financial Data & Stock Exchanges,x2_Food Distribution,x2_Footwear & Accessories,"x2_Furnishings, Fixtures & Appliances",x2_Gold,x2_Grocery Stores,x2_Healthcare Plans,x2_Home Improvement Retail,x2_Household & Personal Products,x2_Industrial Distribution,x2_Information Technology Services,x2_Insurance Brokers,x2_Insurance—Diversified,x2_Insurance—Life,x2_Insurance—Property & Casualty,x2_Insurance—Reinsurance,x2_Insurance—Specialty,x2_Integrated Freight & Logistics,x2_Internet Content & Information,x2_Internet Retail,x2_Leisure,x2_Lodging,x2_Medical Care Facilities,x2_Medical Devices,x2_Medical Distribution,x2_Medical Instruments & Supplies,x2_Oil & Gas E&P,x2_Oil & Gas Equipment & Services,x2_Oil & Gas Integrated,x2_Oil & Gas Midstream,x2_Oil & Gas Refining & Marketing,x2_Packaged Foods,x2_Packaging & Containers,x2_Personal Services,x2_Pharmaceutical Retailers,x2_REIT—Diversified,x2_REIT—Healthcare Facilities,x2_REIT—Hotel & Motel,x2_REIT—Industrial,x2_REIT—Office,x2_REIT—Residential,x2_REIT—Retail,x2_REIT—Specialty,x2_Railroads,x2_Real Estate Services,x2_Rental & Leasing Services,x2_Residential Construction,x2_Resorts & Casinos,x2_Restaurants,x2_Scientific & Technical Instruments,x2_Security & Protection Services,x2_Semiconductor Equipment & Materials,x2_Semiconductors,x2_Software—Application,x2_Software—Infrastructure,x2_Solar,x2_Specialty Business Services,x2_Specialty Chemicals,x2_Specialty Industrial Machinery,x2_Specialty Retail,x2_Staffing & Employment Services,x2_Steel,x2_Telecom Services,x2_Tobacco,x2_Tools & Accessories,x2_Travel Services,x2_Trucking,x2_Utilities—Diversified,x2_Utilities—Independent Power Producers,x2_Utilities—Regulated Electric,x2_Utilities—Regulated Gas,x2_Utilities—Regulated Water,x2_Utilities—Renewable,x2_Waste Management,x2_nan,x3_AL,x3_AR,x3_AZ,x3_CA,x3_CO,x3_CT,x3_DC,x3_DE,x3_FL,x3_GA,x3_IA,x3_ID,x3_IL,x3_IN,x3_KY,x3_LA,x3_MA,x3_MD,x3_ME,x3_MI,x3_MN,x3_MO,x3_NC,x3_NE,x3_NJ,x3_NV,x3_NY,x3_OH,x3_OK,x3_OR,x3_PA,x3_RI,x3_TN,x3_TX,x3_UT,x3_VA,x3_WA,x3_WI,x3_nan,x4_Bermuda,x4_Ireland,x4_Israel,x4_Netherlands,x4_Switzerland,x4_United Kingdom,x4_United States,x4_nan,Symbol,Currentprice,Marketcap,Ebitda,Revenuegrowth,Fulltimeemployees,Weight,Adj Close,Close,High,Low,Open,Volume,S&P500
3,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,GOOG,98.71,1281502609408,9.688700e+10,0.126,174014.0,0.039804,18.716845,18.716845,18.891689,18.687206,18.752213,95713418.0,1432.84
498,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [47]:
googY = goog_df["Close"]
googX = goog_df.drop(["Close", "Symbol", "Adj Close", "High", "Low", "Open"], axis = 1)

## linear regression

In [48]:
regr = linear_model.LinearRegression()

In [49]:
kf = model_selection.KFold(n_splits=5, shuffle=True)

r2scores = []
RMSEscores = []

for train_index, test_index in kf.split(goog_df):
    X_train, X_test = googX.iloc[train_index], googX.iloc[test_index]
    y_train, y_test = googY.iloc[train_index], googY.iloc[test_index]
    regr.fit(X_train, y_train)
    y_pred = regr.predict(X_test)
    r2scores.append(metrics.r2_score(y_test, y_pred))
    RMSEscores.append(math.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print("r2=  {}".format(metrics.r2_score(y_test, y_pred)))
    print("root MSE= {}".format(math.sqrt(metrics.mean_squared_error(y_test, y_pred))))
    print()

print("average r2=  {}".format(np.mean(r2scores)))
print("std=  {}".format(np.std(r2scores)))
print("average root MSE=  {}".format(np.mean(RMSEscores)))
print("std=  {}".format(np.std(RMSEscores)))
print()

print("predicted:")
print(y_pred[-25:])
print("ground truth:")
print(googY[-25:].values)

r2=  0.9705608435598085
root MSE= 6.064301230997427

r2=  0.9742996172819902
root MSE= 5.676057656359585

r2=  0.9726442491874315
root MSE= 5.7270526170456115

r2=  0.970488425501365
root MSE= 5.752085088914114

r2=  0.9737464052794248
root MSE= 6.031444860205041

average r2=  0.972347908162004
std=  0.001581395354581518
average root MSE=  5.850188290704356
std=  0.16358875593041408

predicted:
[103.83608234 102.2671566  101.63789558 103.34934485 108.11848414
 109.43971515 111.74273276 114.27905309 115.93433666 114.73810279
 121.33484256 114.08953035 114.69542015 106.23555994 105.58115542
 108.6600312  109.6333102  113.7245698  107.22494543  97.22165084
  98.19577003  94.8180753   92.89953077 100.96760988 100.36673319]
ground truth:
[107.48000336 110.48000336 109.41999817 111.77999878 111.87000275
 105.30999756 105.87000275 103.90000153 103.62999725 103.84999847
 101.83000183 100.01000214 100.56999969  99.16999817  98.80999756
  98.08999634 100.73999786  98.08999634  96.15000153  99.30

## kNN regression 

### k=1

In [50]:
knn_regr = neighbors.KNeighborsRegressor(n_neighbors=1, weights='uniform')

In [51]:
kf = model_selection.KFold(n_splits=5, shuffle=True)

r2scores = []
RMSEscores = []

for train_index, test_index in kf.split(goog_df):
    X_train, X_test = googX.iloc[train_index], googX.iloc[test_index]
    y_train, y_test = googY.iloc[train_index], googY.iloc[test_index]
    knn_regr.fit(X_train, y_train)
    y_pred = knn_regr.predict(X_test)
    r2scores.append(metrics.r2_score(y_test, y_pred))
    RMSEscores.append(math.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print("r2=  {}".format(metrics.r2_score(y_test, y_pred)))
    print("root MSE= {}".format(math.sqrt(metrics.mean_squared_error(y_test, y_pred))))
    print()

print("average r2=  {}".format(np.mean(r2scores)))
print("std=  {}".format(np.std(r2scores)))
print("average root MSE=  {}".format(np.mean(RMSEscores)))
print("std=  {}".format(np.std(RMSEscores)))
print()

print("predicted:")
print(y_pred[-25:])
print("ground truth:")
print(googY[-25:].values)

r2=  -0.5015359443053335
root MSE= 42.6550398552682

r2=  -0.26328701035464874
root MSE= 40.330623754722964

r2=  -0.723594684622487
root MSE= 45.32147583495339

r2=  -0.40008466485185523
root MSE= 39.28768650445123

r2=  -0.5093704776769263
root MSE= 46.10061223167174

average r2=  -0.47957455636225016
std=  0.15100653488762933
average root MSE=  42.739087636213505
std=  2.6716477494279776

predicted:
[ 53.98450089  82.51049805  60.81700134  26.96496773 119.93450165
  40.9754982   41.76200104 141.45300293  52.18299866  36.56150055
  64.16249847  27.64958763  71.4865036   74.01599884  38.35250092
  57.79050064  61.97050095 113.16100311  41.85850143  46.52999878
  62.44200134 118.22000122 112.74199677  62.79999924  47.66350174]
ground truth:
[107.48000336 110.48000336 109.41999817 111.77999878 111.87000275
 105.30999756 105.87000275 103.90000153 103.62999725 103.84999847
 101.83000183 100.01000214 100.56999969  99.16999817  98.80999756
  98.08999634 100.73999786  98.08999634  96.1500015

### k=5

In [52]:
knn_regr = neighbors.KNeighborsRegressor(n_neighbors=5, weights='uniform')

In [53]:
kf = model_selection.KFold(n_splits=5, shuffle=True)

r2scores = []
RMSEscores = []

for train_index, test_index in kf.split(goog_df):
    X_train, X_test = googX.iloc[train_index], googX.iloc[test_index]
    y_train, y_test = googY.iloc[train_index], googY.iloc[test_index]
    knn_regr.fit(X_train, y_train)
    y_pred = knn_regr.predict(X_test)
    r2scores.append(metrics.r2_score(y_test, y_pred))
    RMSEscores.append(math.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print("r2=  {}".format(metrics.r2_score(y_test, y_pred)))
    print("root MSE= {}".format(math.sqrt(metrics.mean_squared_error(y_test, y_pred))))
    print()

print("average r2=  {}".format(np.mean(r2scores)))
print("std=  {}".format(np.std(r2scores)))
print("average root MSE=  {}".format(np.mean(RMSEscores)))
print("std=  {}".format(np.std(RMSEscores)))
print()

print("predicted:")
print(y_pred[-25:])
print("ground truth:")
print(googY[-25:].values)

r2=  0.12278155672980551
root MSE= 32.609613628139186

r2=  -0.03675075944863537
root MSE= 34.68234598584408

r2=  0.09777436049880361
root MSE= 35.39258307481348

r2=  0.1108787457962973
root MSE= 32.836974450040444

r2=  0.07842257207222636
root MSE= 33.783544499762904

average r2=  0.07462129512969948
std=  0.05759906632856268
average root MSE=  33.86101232772002
std=  1.0621630266511215

predicted:
[ 50.45591164  53.51659431  53.98130875  58.50950012  29.07360077
  77.91250229  58.4928997   57.74919968  70.59950027  56.39809952
  55.7504879   41.5267437   54.53603401  49.64902306  78.10875893
  82.89839935  69.64440002  37.49538345  99.90319824 112.62289886
  47.95479012 106.04369965  82.60119934  81.90189972  66.66489944]
ground truth:
[107.48000336 110.48000336 109.41999817 111.77999878 111.87000275
 105.30999756 105.87000275 103.90000153 103.62999725 103.84999847
 101.83000183 100.01000214 100.56999969  99.16999817  98.80999756
  98.08999634 100.73999786  98.08999634  96.1500015

### k=11

In [54]:
knn_regr = neighbors.KNeighborsRegressor(n_neighbors=11, weights='uniform')

In [55]:
kf = model_selection.KFold(n_splits=5, shuffle=True)

r2scores = []
RMSEscores = []

for train_index, test_index in kf.split(goog_df):
    X_train, X_test = googX.iloc[train_index], googX.iloc[test_index]
    y_train, y_test = googY.iloc[train_index], googY.iloc[test_index]
    knn_regr.fit(X_train, y_train)
    y_pred = knn_regr.predict(X_test)
    r2scores.append(metrics.r2_score(y_test, y_pred))
    RMSEscores.append(math.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print("r2=  {}".format(metrics.r2_score(y_test, y_pred)))
    print("root MSE= {}".format(math.sqrt(metrics.mean_squared_error(y_test, y_pred))))
    print()

print("average r2=  {}".format(np.mean(r2scores)))
print("std=  {}".format(np.std(r2scores)))
print("average root MSE=  {}".format(np.mean(RMSEscores)))
print("std=  {}".format(np.std(RMSEscores)))
print()

print("predicted:")
print(y_pred[-25:])
print("ground truth:")
print(googY[-25:].values)

r2=  0.17733048459839706
root MSE= 33.27155749208573

r2=  0.13813481395736327
root MSE= 33.98781117623855

r2=  0.14122900225735135
root MSE= 28.855460892866226

r2=  0.1642396567701454
root MSE= 33.17557482226677

r2=  0.09827833095370875
root MSE= 33.15054105580834

average r2=  0.14384245770739318
std=  0.02702957886483091
average root MSE=  32.488189087853115
std=  1.842305939886517

predicted:
[ 42.2140229   62.67993927 104.45286317  66.60172757  45.44323037
  54.70216699  49.41957664  73.45909015  61.34339281  71.00571927
  65.99499997  72.39885174  63.11995437  55.71396047  67.77289547
  45.91779917  98.74608959  98.85854444  44.71367663  74.77489281
  90.42097213  76.46384291  32.78946738  90.42097213  65.64877319]
ground truth:
[107.48000336 110.48000336 109.41999817 111.77999878 111.87000275
 105.30999756 105.87000275 103.90000153 103.62999725 103.84999847
 101.83000183 100.01000214 100.56999969  99.16999817  98.80999756
  98.08999634 100.73999786  98.08999634  96.15000153  

## neural network

In [59]:
X = googX.to_numpy()
y = googY.to_numpy()

### activation='relu'

layers=1, size=100

In [60]:
kfold = model_selection.KFold(5, shuffle=True, random_state=2)

r2= []
RMSE = []

for train_idx, test_idx in kfold.split(goog_df):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    mlp_regr = MLPRegressor(hidden_layer_sizes=(100), max_iter=2000, activation="relu", random_state=2)
    X_scaler = preprocessing.MinMaxScaler()
    
    X_train = X_scaler.fit_transform(X_train)    
    
    mlp_regr.fit(X_train, y_train)
    
    X_test = X_scaler.transform(X_test)
    y_pred = mlp_regr.predict(X_test)
    
    r2 += [metrics.r2_score(y_test, y_pred)]
    RMSE += [math.sqrt(metrics.mean_squared_error(y_test, y_pred))]

print("r2    = {:.4f} ±{:.4f}".format(np.mean(r2), np.std(r2)))
print("RMSE = {:.4f} ±{:.4f}".format(np.mean(RMSE), np.std(RMSE)))

r2    = 0.9721 ±0.0021
RMSE = 5.8683 ±0.1289


layers=2, size=100

In [61]:
kfold = model_selection.KFold(5, shuffle=True, random_state=2)

r2= []
RMSE = []

for train_idx, test_idx in kfold.split(goog_df):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    mlp_regr = MLPRegressor(hidden_layer_sizes=(100, 100), max_iter=2000, activation="relu", random_state=2)
    X_scaler = preprocessing.MinMaxScaler()
    
    X_train = X_scaler.fit_transform(X_train)    
    
    mlp_regr.fit(X_train, y_train)
    
    X_test = X_scaler.transform(X_test)
    y_pred = mlp_regr.predict(X_test)
    
    r2 += [metrics.r2_score(y_test, y_pred)]
    RMSE += [math.sqrt(metrics.mean_squared_error(y_test, y_pred))]

print("r2    = {:.4f} ±{:.4f}".format(np.mean(r2), np.std(r2)))
print("RMSE = {:.4f} ±{:.4f}".format(np.mean(RMSE), np.std(RMSE)))

r2    = 0.9884 ±0.0012
RMSE = 3.7888 ±0.1209


### activation='tanh'

layers=1, size=100

In [30]:
kfold = model_selection.KFold(5, shuffle=True, random_state=2)

r2= []
RMSE = []

for train_idx, test_idx in kfold.split(goog_df):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    mlp_regr = MLPRegressor(hidden_layer_sizes=(100), max_iter=2000, activation="tanh", random_state=2)
    X_scaler = preprocessing.MinMaxScaler()
    
    X_train = X_scaler.fit_transform(X_train)    
    
    mlp_regr.fit(X_train, y_train)
    
    X_test = X_scaler.transform(X_test)
    y_pred = mlp_regr.predict(X_test)
    
    r2 += [metrics.r2_score(y_test, y_pred)]
    RMSE += [math.sqrt(metrics.mean_squared_error(y_test, y_pred))]

print("r2    = {:.4f} ±{:.4f}".format(np.mean(r2), np.std(r2)))
print("RMSE = {:.4f} ±{:.4f}".format(np.mean(RMSE), np.std(RMSE)))

r2    = 0.9877 ±0.0012
RMSE = 3.8905 ±0.1502


layers=2, size=100

In [31]:
kfold = model_selection.KFold(5, shuffle=True, random_state=2)

r2= []
RMSE = []

for train_idx, test_idx in kfold.split(goog_df):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    mlp_regr = MLPRegressor(hidden_layer_sizes=(100, 100), max_iter=2000, activation="tanh", random_state=2)
    
    X_train = X_scaler.fit_transform(X_train)    
    
    mlp_regr.fit(X_train, y_train)
    
    X_test = X_scaler.transform(X_test)
    y_pred = mlp_regr.predict(X_test)
    
    r2 += [metrics.r2_score(y_test, y_pred)]
    RMSE += [math.sqrt(metrics.mean_squared_error(y_test, y_pred))]

print("r2    = {:.4f} ±{:.4f}".format(np.mean(r2), np.std(r2)))
print("RMSE = {:.4f} ±{:.4f}".format(np.mean(RMSE), np.std(RMSE)))

r2    = 0.9891 ±0.0013
RMSE = 3.6719 ±0.1346


# 

# 

# TWTR stock 

In [32]:
twtr_df = snp_df[snp_df['Symbol'] == 'TWTR']
twtrY = twtr_df["Close"]
twtrX = twtr_df.drop(["Close", "Symbol", "Adj Close", "High", "Low", "Open"], axis = 1)

### linear regression

In [33]:
twtr_regr = linear_model.LinearRegression()

In [34]:
kf = model_selection.KFold(n_splits=5, shuffle=True)

r2scores = []
MSEscores = []

for train_index, test_index in kf.split(twtr_df):
    X_train, X_test = twtrX.iloc[train_index], twtrX.iloc[test_index]
    y_train, y_test = twtrY.iloc[train_index], twtrY.iloc[test_index]
    twtr_regr.fit(X_train, y_train)
    y_pred = twtr_regr.predict(X_test)
    r2scores.append(metrics.r2_score(y_test, y_pred))
    MSEscores.append(math.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print("r2=  {}".format(metrics.r2_score(y_test, y_pred)))
    print("root MSE= {}".format(math.sqrt(metrics.mean_squared_error(y_test, y_pred))))
    print()

print("average r2=  {}".format(np.mean(r2scores)))
print("std=  {}".format(np.std(r2scores)))
print("average root MSE=  {}".format(np.mean(MSEscores)))
print("std=  {}".format(np.std(MSEscores)))
print()

print("predicted:")
print(y_pred[-25:])
print("ground truth:")
print(twtrY[-25:].values)

r2=  0.2487056044283691
root MSE= 12.368300632100834

r2=  0.30067742005448816
root MSE= 11.86632948626179

r2=  0.22035669731659413
root MSE= 12.330308417119292

r2=  0.22858166912571676
root MSE= 11.931440523166614

r2=  0.2222459982130801
root MSE= 12.637620888051513

average r2=  0.24411347782764964
std=  0.030014581634451212
average root MSE=  12.226799989340009
std=  0.28868198253056615

predicted:
[49.94454524 48.66946167 49.39074163 49.15869626 50.41241827 42.20185763
 45.34579863 44.64929162 47.72161076 46.57104761 46.93765149 47.11849944
 44.13026624 44.67749108 44.92862076 45.13852473 45.83964974 47.17841008
 47.69647667 47.53749012 48.94107804 46.65408682 44.58979625 43.82017861
 42.5387796 ]
ground truth:
[38.65000153 41.20000076 41.84999847 42.18999863 41.40999985 41.74000168
 41.90000153 42.13999939 41.45000076 41.65999985 41.68000031 41.27000046
 41.40000153 41.58000183 41.52000046 42.09000015 43.25       42.74000168
 43.84000015 42.54000092 52.         51.29999924 49.3

### kNN regression

### k=1

In [35]:
knn_regr = neighbors.KNeighborsRegressor(n_neighbors=1, weights='uniform')

In [36]:
kf = model_selection.KFold(n_splits=5, shuffle=True)

r2scores = []
RMSEscores = []

for train_index, test_index in kf.split(twtr_df):
    X_train, X_test = twtrX.iloc[train_index], twtrX.iloc[test_index]
    y_train, y_test = twtrY.iloc[train_index], twtrY.iloc[test_index]
    knn_regr.fit(X_train, y_train)
    y_pred = knn_regr.predict(X_test)
    r2scores.append(metrics.r2_score(y_test, y_pred))
    RMSEscores.append(math.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print("r2=  {}".format(metrics.r2_score(y_test, y_pred)))
    print("root MSE= {}".format(math.sqrt(metrics.mean_squared_error(y_test, y_pred))))
    print()

print("average r2=  {}".format(np.mean(r2scores)))
print("std=  {}".format(np.std(r2scores)))
print("average root MSE=  {}".format(np.mean(RMSEscores)))
print("std=  {}".format(np.std(RMSEscores)))
print()

print("predicted:")
print(y_pred[-25:])
print("ground truth:")
print(twtrY[-25:].values)

r2=  -0.791833962773117
root MSE= 19.002756796573067

r2=  -0.8819354956401018
root MSE= 18.65409255083401

r2=  -0.8479674349446575
root MSE= 19.60588219738036

r2=  -1.129465608275487
root MSE= 19.666403979895513

r2=  -1.0008214604390901
root MSE= 20.60244346376594

average r2=  -0.9304047924144907
std=  0.12077316010674193
average root MSE=  19.506315797689776
std=  0.6658669062889221

predicted:
[20.86000061 47.95999908 18.70000076 16.07999992 34.93000031 37.65999985
 18.62999916 39.95000076 17.60000038 64.98000336 64.66000366 69.25
 25.53000069 24.01000023 41.         54.90999985 16.90999985 18.28000069
 37.70000076 41.40999985 37.70000076 37.47000122 37.13999939 25.18000031
 63.68000031]
ground truth:
[38.65000153 41.20000076 41.84999847 42.18999863 41.40999985 41.74000168
 41.90000153 42.13999939 41.45000076 41.65999985 41.68000031 41.27000046
 41.40000153 41.58000183 41.52000046 42.09000015 43.25       42.74000168
 43.84000015 42.54000092 52.         51.29999924 49.38999939 49

### k=5

In [37]:
knn_regr = neighbors.KNeighborsRegressor(n_neighbors=5, weights='uniform')

In [38]:
kf = model_selection.KFold(n_splits=5, shuffle=True)

r2scores = []
RMSEscores = []

for train_index, test_index in kf.split(twtr_df):
    X_train, X_test = twtrX.iloc[train_index], twtrX.iloc[test_index]
    y_train, y_test = twtrY.iloc[train_index], twtrY.iloc[test_index]
    knn_regr.fit(X_train, y_train)
    y_pred = knn_regr.predict(X_test)
    r2scores.append(metrics.r2_score(y_test, y_pred))
    RMSEscores.append(math.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print("r2=  {}".format(metrics.r2_score(y_test, y_pred)))
    print("root MSE= {}".format(math.sqrt(metrics.mean_squared_error(y_test, y_pred))))
    print()

print("average r2=  {}".format(np.mean(r2scores)))
print("std=  {}".format(np.std(r2scores)))
print("average root MSE=  {}".format(np.mean(RMSEscores)))
print("std=  {}".format(np.std(RMSEscores)))
print()

print("predicted:")
print(y_pred[-25:])
print("ground truth:")
print(twtrY[-25:].values)

r2=  -0.22368404772856976
root MSE= 15.28433985649056

r2=  -0.18662584683734473
root MSE= 15.292970502506316

r2=  -0.12173896724021782
root MSE= 14.14132191304396

r2=  -0.15616688384919097
root MSE= 15.950104831019813

r2=  -0.092174179430498
root MSE= 14.929771109524426

average r2=  -0.15607798501716424
std=  0.046403930847439094
average root MSE=  15.119701642517015
std=  0.589914874744589

predicted:
[27.7920002  28.43999996 38.1920002  29.3420002  29.47400036 42.3720005
 52.35799942 42.39400063 34.46200104 37.88600006 33.50399971 31.59199944
 41.72400055 38.10200005 37.38199997 49.42200012 28.01199989 41.92599945
 33.73800049 61.23799973 41.79000015 37.08999977 42.8560009  42.8560009
 36.67400017]
ground truth:
[38.65000153 41.20000076 41.84999847 42.18999863 41.40999985 41.74000168
 41.90000153 42.13999939 41.45000076 41.65999985 41.68000031 41.27000046
 41.40000153 41.58000183 41.52000046 42.09000015 43.25       42.74000168
 43.84000015 42.54000092 52.         51.29999924 49.

### k=11

In [39]:
knn_regr = neighbors.KNeighborsRegressor(n_neighbors=11, weights='uniform')

In [40]:
kf = model_selection.KFold(n_splits=5, shuffle=True)

r2scores = []
RMSEscores = []

for train_index, test_index in kf.split(twtr_df):
    X_train, X_test = twtrX.iloc[train_index], twtrX.iloc[test_index]
    y_train, y_test = twtrY.iloc[train_index], twtrY.iloc[test_index]
    knn_regr.fit(X_train, y_train)
    y_pred = knn_regr.predict(X_test)
    r2scores.append(metrics.r2_score(y_test, y_pred))
    RMSEscores.append(math.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print("r2=  {}".format(metrics.r2_score(y_test, y_pred)))
    print("root MSE= {}".format(math.sqrt(metrics.mean_squared_error(y_test, y_pred))))
    print()

print("average r2=  {}".format(np.mean(r2scores)))
print("std=  {}".format(np.std(r2scores)))
print("average root MSE=  {}".format(np.mean(RMSEscores)))
print("std=  {}".format(np.std(RMSEscores)))
print()

print("predicted:")
print(y_pred[-25:])
print("ground truth:")
print(twtrY[-25:].values)

r2=  -0.018765101991531363
root MSE= 14.034031984972833

r2=  0.0005038972258778474
root MSE= 13.676459677964207

r2=  -0.09671667749694701
root MSE= 14.538986973667384

r2=  -0.04062145486398072
root MSE= 14.934227937689126

r2=  -0.08408732819959419
root MSE= 14.786790642248027

average r2=  -0.04793733306523509
std=  0.03724873292661527
average root MSE=  14.394099443308315
std=  0.471506727836552

predicted:
[34.78181874 29.73727278 35.31818208 30.40909117 33.35454559 44.48181829
 34.45090918 44.38545461 39.18909125 43.80363638 32.11272717 37.49181851
 33.0727272  47.30909105 37.23181846 42.99272737 38.26818189 36.35363622
 42.63545435 43.54636305 41.34909058 42.08545399 35.31818208 29.58727264
 35.20181829]
ground truth:
[38.65000153 41.20000076 41.84999847 42.18999863 41.40999985 41.74000168
 41.90000153 42.13999939 41.45000076 41.65999985 41.68000031 41.27000046
 41.40000153 41.58000183 41.52000046 42.09000015 43.25       42.74000168
 43.84000015 42.54000092 52.         51.29999

## neural network

In [41]:
X = twtrX.to_numpy()
y = twtrY.to_numpy()

### activation='relu'

layers=1, size=100

In [42]:
kfold = model_selection.KFold(5, shuffle=True, random_state=2)

r2= []
RMSE = []

for train_idx, test_idx in kfold.split(twtr_df):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    mlp_regr = MLPRegressor(hidden_layer_sizes=(100), max_iter=2000, activation="relu", random_state=2)
    X_scaler = preprocessing.MinMaxScaler()
    
    X_train = X_scaler.fit_transform(X_train)    
    
    mlp_regr.fit(X_train, y_train)
    
    X_test = X_scaler.transform(X_test)
    y_pred = mlp_regr.predict(X_test)
    
    r2 += [metrics.r2_score(y_test, y_pred)]
    RMSE += [math.sqrt(metrics.mean_squared_error(y_test, y_pred))]

print("r2    = {:.4f} ±{:.4f}".format(np.mean(r2), np.std(r2)))
print("RMSE = {:.4f} ±{:.4f}".format(np.mean(RMSE), np.std(RMSE)))

r2    = 0.2429 ±0.0387
RMSE = 12.2331 ±0.1913


layers=2, size=100 

In [43]:
kfold = model_selection.KFold(5, shuffle=True, random_state=2)

r2= []
RMSE = []

for train_idx, test_idx in kfold.split(twtr_df):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    mlp_regr = MLPRegressor(hidden_layer_sizes=(100, 100), max_iter=2000, activation="relu", random_state=2)
    X_scaler = preprocessing.MinMaxScaler()
    
    X_train = X_scaler.fit_transform(X_train)    
    
    mlp_regr.fit(X_train, y_train)
    
    X_test = X_scaler.transform(X_test)
    y_pred = mlp_regr.predict(X_test)
    
    r2 += [metrics.r2_score(y_test, y_pred)]
    RMSE += [math.sqrt(metrics.mean_squared_error(y_test, y_pred))]

print("r2    = {:.4f} ±{:.4f}".format(np.mean(r2), np.std(r2)))
print("RMSE = {:.4f} ±{:.4f}".format(np.mean(RMSE), np.std(RMSE)))

r2    = 0.5448 ±0.0120
RMSE = 9.4928 ±0.2625


### activation='tanh'

layers=1, size=100

In [44]:
kfold = model_selection.KFold(5, shuffle=True, random_state=2)

r2= []
RMSE = []

for train_idx, test_idx in kfold.split(twtr_df):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    mlp_regr = MLPRegressor(hidden_layer_sizes=(100), max_iter=2000, activation="tanh", random_state=2)
    X_scaler = preprocessing.MinMaxScaler()
    
    X_train = X_scaler.fit_transform(X_train)    
    
    mlp_regr.fit(X_train, y_train)
    
    X_test = X_scaler.transform(X_test)
    y_pred = mlp_regr.predict(X_test)
    
    r2 += [metrics.r2_score(y_test, y_pred)]
    RMSE += [math.sqrt(metrics.mean_squared_error(y_test, y_pred))]

print("r2    = {:.4f} ±{:.4f}".format(np.mean(r2), np.std(r2)))
print("RMSE = {:.4f} ±{:.4f}".format(np.mean(RMSE), np.std(RMSE)))

/Users/caitlinhermann/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/caitlinhermann/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/caitlinhermann/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/caitlinhermann/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
 

r2    = 0.3924 ±0.0326
RMSE = 10.9579 ±0.1478


/Users/caitlinhermann/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(


layers=2, size=100

In [45]:
kfold = model_selection.KFold(5, shuffle=True, random_state=2)

r2= []
RMSE = []

for train_idx, test_idx in kfold.split(twtr_df):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    mlp_regr = MLPRegressor(hidden_layer_sizes=(100, 100), max_iter=2000, activation="tanh", random_state=2)
    X_scaler = preprocessing.MinMaxScaler()
    
    X_train = X_scaler.fit_transform(X_train)    
    
    mlp_regr.fit(X_train, y_train)
    
    X_test = X_scaler.transform(X_test)
    y_pred = mlp_regr.predict(X_test)
    
    r2 += [metrics.r2_score(y_test, y_pred)]
    RMSE += [math.sqrt(metrics.mean_squared_error(y_test, y_pred))]

print("r2    = {:.4f} ±{:.4f}".format(np.mean(r2), np.std(r2)))
print("RMSE = {:.4f} ±{:.4f}".format(np.mean(RMSE), np.std(RMSE)))

r2    = 0.6122 ±0.0270
RMSE = 8.7514 ±0.2216
